In [1]:
import os
from langchain_nvidia_ai_endpoints import NVIDIARerank, NVIDIAEmbeddings
from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from astrapy import DataAPIClient
from dotenv import load_dotenv
from astrapy.info import CreateTableDefinition, ColumnType
from astrapy.data_types import DataAPIMap, DataAPIVector


load_dotenv()


ASTRA_DB_API_ENDPOINT = os.getenv("ASTRA_DB_API_ENDPOINT")
ASTRA_DB_APPLICATION_TOKEN = os.getenv("ASTRA_DB_APPLICATION_TOKEN")
NVIDIA_API_KEY = os.getenv("NVIDIA_API_KEY")


client = DataAPIClient(ASTRA_DB_APPLICATION_TOKEN)
database = client.get_database(ASTRA_DB_API_ENDPOINT)
embedv1 = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", api_key=NVIDIA_API_KEY)
mistral = NVIDIAEmbeddings(model="nvidia/nv-embedqa-mistral-7b-v2", api_key=NVIDIA_API_KEY)
documents = TextLoader("moby10b.txt").load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)
texts = text_splitter.split_documents(documents)



In [ ]:
schema = (
    CreateTableDefinition.builder()
    .add_column("chunk_id", ColumnType.INT)
    .add_column("document_id", ColumnType.TEXT)
    .add_vector_column("nv_embed_v1", dimension=4096)
    .add_vector_column("nv_embedqa_mistral_7b_v2", dimension=4096)
    .add_map_column("metadata", ColumnType.TEXT, ColumnType.TEXT)
    .add_column("content", ColumnType.TEXT)
    .add_partition_by(["chunk_id"])
    .build()
)


table = database.create_table(
    "multivector",
    definition=schema,
)

In [3]:
table = database.get_table("multivector")


table.create_vector_index("nv_embed_v1_index", column="nv_embed_v1", if_not_exists=True)
table.create_vector_index("nv_embedqa_mistral_7b_v2_index", column="nv_embedqa_mistral_7b_v2", if_not_exists=True)
table.create_index("document_id_index", column="document_id", if_not_exists=True)

In [ ]:
for i, chunk in enumerate(texts):
    table.insert_one(
        {
            "chunk_id": i,
            "document_id": chunk.metadata["source"],
            "nv_embed_v1": DataAPIVector(embedv1.embed_query(chunk.page_content)),
            "nv_embedqa_mistral_7b_v2": DataAPIVector(mistral.embed_query(chunk.page_content)),
            "metadata": DataAPIMap(chunk.metadata),
            "content": chunk.page_content
        }
    )

In [4]:
query = "What are the main character's thoughts on whether or not whales are fish?"
query0 = embedv1.embed_query(query)
query1 = mistral.embed_query(query)

In [25]:
cursor = table.find(filter={"document_id": "moby10b.txt"}, 
                    sort={"nv_embed_v1": DataAPIVector(query0)},
                    limit=10,
                    projection={"chunk_id": True, "content": True},
                    include_similarity=True)

embedv1_results = [row for row in cursor]


In [26]:
cursor = table.find(filter={"document_id": "moby10b.txt"}, 
                    sort={"nv_embedqa_mistral_7b_v2": DataAPIVector(query1)},
                    limit=10,
                    projection={"chunk_id": True, "content": True},
                    include_similarity=True)

mistral_results = [row for row in cursor]

In [ ]:
from langchain_core.documents import Document


all_docs = [Document(doc['content']) for doc in embedv1_results] + [Document(doc['content']) for doc in mistral_results]
reranker = NVIDIARerank(base_url="https://integrate.api.nvidia.com/v1", api_key=NVIDIA_API_KEY)
reranker.top_n = 5
docs = reranker.compress_documents(query=query, documents=all_docs)
docs